In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [2]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

100.0%
100.0%
100.0%
100.0%


In [4]:
num_classes = len(train_data.classes)
print(num_classes)

10


In [6]:
dataloader_train = DataLoader(
    train_data,
    shuffle = True,
    batch_size = 1,
)

dataloader_test = DataLoader(
    test_data,
    shuffle = True,
    batch_size = 1,
)

In [7]:
image, label = next(iter(dataloader_train))
print(image.shape)

torch.Size([1, 1, 28, 28])


In [8]:
# Start coding here
# Use as many cells as you need
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
    nn.ELU(),
    nn.MaxPool2d(kernel_size = 2),
    nn.Conv2d(32,64, kernel_size = 3, padding = 1),
    nn.ELU(),
    nn.MaxPool2d(kernel_size = 2),
    nn.Flatten(),
    nn.Linear(64 * 7 * 7, num_classes)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(2):
    for images, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

KeyboardInterrupt: 

In [ ]:
precision_ = Precision(
    task = "multiclass", num_classes= num_classes, average = None
)

recall_ = Recall(
    task = "multiclass", num_classes= num_classes, average = None
)

accuracy_ = Accuracy(
    task = "multiclass", num_classes= num_classes
)

predictions = []
true_labels = []

model.eval()
with torch.no_grad():
    for images, labels in dataloader_test:
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.append(preds.item())
        true_labels.append(labels.item())
        precision_(preds, labels)
        recall_(preds, labels)
        accuracy_(preds, labels)
        
precision = precision_.compute().tolist()
print(f"Precision per class: {precision}")
recall = recall_.compute().item()
print(f"Recall per class: {recall}")
accuracy = accuracy_.compute().tolist()
print(f"Overall accuracy: {accuracy}")    
